In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.nn as nn

In [2]:

A = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
B = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)


add = tf.add(A, B)
sub = tf.subtract(A, B)
mul = tf.matmul(A, B)
transpose = tf.transpose(A)
inv = tf.linalg.inv(A)

print("Addition Result:\n", add.numpy())
print("Subtraction Result:\n", sub.numpy())
print("Multiplication Result:\n", mul.numpy())
print("Transpose Result:\n", transpose.numpy())
print("Inverse Result:\n", inv.numpy())

Addition Result:
 [[ 6.  8.]
 [10. 12.]]
Subtraction Result:
 [[-4. -4.]
 [-4. -4.]]
Multiplication Result:
 [[19. 22.]
 [43. 50.]]
Transpose Result:
 [[1. 3.]
 [2. 4.]]
Inverse Result:
 [[-2.   1. ]
 [ 1.5 -0.5]]


In [3]:
def perceptron(inputs, weights, bias):
    activation = np.dot(inputs, weights) + bias
    return 1 if activation >= 0 else 0
gates = {
    "AND": {"w": [1, 1], "b": -1.5},
    "OR":  {"w": [1, 1], "b": -0.5}
}
test_inputs = [[0,0], [0,1], [1,0], [1,1]]
for name, params in gates.items():
    print(f"--- {name} Gate ---")
    for x in test_inputs:
        res = perceptron(x, params["w"], params["b"])
        print(f"Input: {x} Output: {res}")

--- AND Gate ---
Input: [0, 0] Output: 0
Input: [0, 1] Output: 0
Input: [1, 0] Output: 0
Input: [1, 1] Output: 1
--- OR Gate ---
Input: [0, 0] Output: 0
Input: [0, 1] Output: 1
Input: [1, 0] Output: 1
Input: [1, 1] Output: 1


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# ======================
# 1. XOR Dataset
# ======================
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# ======================
# 2. Neural Network Model
# ======================
class XORNet(nn.Module):# nn.Module is the base class for all neural network modules in PyTorch
    def __init__(self):
        super(XORNet, self).__init__()
        self.fc1 = nn.Linear(2, 2)  # Input to hidden layer
        self.fc2 = nn.Linear(2, 1)   # Hidden to output layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = XORNet()

# ======================
# 3. Training Setup
# ======================
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# ======================
# 4. Training Loop
# ======================
epochs = 10000
for epoch in range(epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# ======================
# 5. Evaluation
# ======================
with torch.no_grad():
    predictions = model(X)
    predicted_labels = (predictions > 0.5).float()
    accuracy = (predicted_labels == y).float().mean()

    print("\nFinal Predictions:")
    for i in range(len(X)):
        print(f"Input: {X[i].tolist()}, Predicted: {predictions[i].item():.4f}, Target: {y[i].item()}")

    print(f"\nAccuracy: {accuracy.item()*100:.2f}%")

Epoch [1000/10000], Loss: 0.2138
Epoch [2000/10000], Loss: 0.0100
Epoch [3000/10000], Loss: 0.0049
Epoch [4000/10000], Loss: 0.0032
Epoch [5000/10000], Loss: 0.0024
Epoch [6000/10000], Loss: 0.0019
Epoch [7000/10000], Loss: 0.0016
Epoch [8000/10000], Loss: 0.0014
Epoch [9000/10000], Loss: 0.0012
Epoch [10000/10000], Loss: 0.0010

Final Predictions:
Input: [0.0, 0.0], Predicted: 0.0009, Target: 0.0
Input: [0.0, 1.0], Predicted: 0.9991, Target: 1.0
Input: [1.0, 0.0], Predicted: 0.9991, Target: 1.0
Input: [1.0, 1.0], Predicted: 0.0014, Target: 0.0

Accuracy: 100.00%


In [10]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
df = pd.read_csv('/content/house_price_full+(2) - house_price_full+(2).csv')

x = df[['bedrooms', 'sqft_living']].values.astype('float32')
y = df[['price']].values.astype('float32')
scaler_x = StandardScaler()
scaler_y = StandardScaler()
x_scaled = scaler_x.fit_transform(x)
y_scaled = scaler_y.fit_transform(y)
x_t = torch.tensor(x_scaled, dtype=torch.float32)
y_t = torch.tensor(y_scaled, dtype=torch.float32)
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 2)
        self.layer2 = nn.Linear(2, 1)

    def forward(self, x):
        z = torch.relu(self.layer1(x))
        h2 = self.layer2(z)
        return h2

model = Model()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 25000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(x_t)
    loss = criterion(outputs, y_t)
    loss.backward()
    optimizer.step()

    if epoch % 5000 == 0:
        print("Epoch", epoch, "Loss:", loss.item())
# Predictions
pred_scaled = model(x_t).detach().numpy()
pred_real = scaler_y.inverse_transform(pred_scaled)

for i in range(10):
    print(f"Input: {x[i]} => Predicted: {pred_real[i][0]:.2f}, Target: {y[i][0]:.2f}")

Epoch 0 Loss: 0.7658886909484863
Epoch 5000 Loss: 0.4981619417667389
Epoch 10000 Loss: 0.4981640577316284
Epoch 15000 Loss: 0.4981623888015747
Epoch 20000 Loss: 0.4981609284877777
Input: [   3. 1340.] => Predicted: 365137.00, Target: 313000.00
Input: [   5. 3650.] => Predicted: 917994.56, Target: 2384000.00
Input: [   3. 1930.] => Predicted: 473495.88, Target: 342000.00
Input: [   3. 2000.] => Predicted: 490892.91, Target: 420000.00
Input: [   4. 1940.] => Predicted: 484495.69, Target: 550000.00
Input: [  2. 880.] => Predicted: 340408.25, Target: 490000.00
Input: [   2. 1350.] => Predicted: 426465.56, Target: 335000.00
Input: [   4. 2710.] => Predicted: 675862.94, Target: 482000.00
Input: [   3. 2430.] => Predicted: 597760.31, Target: 452500.00
Input: [   4. 1520.] => Predicted: 380113.53, Target: 640000.00
